In [1]:
from Orch import Bijection, ExecutionType, GemmTopology, GemmNode, generate_dot_source, SliceIR_node, SliceIR_tree
import numpy as np

In [2]:
shape = (4,4,4)
tiler = np.zeros((3,4)).astype(int)
tiler[0,:] = [1,1,1,1]
tiler[1,:] = [4,0,0,0] 
tiler[2,:] = [1,1,1,1] 

bits = 1 
base = 1 
shift = 1
scale = 3
step = 5
length = 4*4*4 

sub_shape = (1,4,1)
sub_tiler = np.zeros((3,4)).astype(int)
sub_tiler[0,:] = [1,0,0,0]
sub_tiler[1,:] = [1,1,1,1]
sub_tiler[2,:] = [1,0,0,0]

sub_length = 4

b1 = Bijection(length, bits, base, shift, scale, step)

b2 = Bijection(sub_length, bits, base, shift, scale, step)

coarse_toplogy = GemmTopology(shape, tiler, (1,0,2), b1)
fine_subtopology = GemmTopology(sub_shape,sub_tiler, (0,2,1), b2)

In [3]:
root = GemmNode(coarse_toplogy, ExecutionType.PARALLEL)

for m in range(4): 
  for n in range(4): 
    root.set_child(root.topology.get_tile_id((m,0,n)), fine_subtopology,sub_exec_type=ExecutionType.SERIAL)

In [4]:
root.print_layout_table()


Node Layout: PARALLEL | Topology: (4, 4, 4)
-------------------------------------------------------------------------------------
ID   | Rank | Slice (M, K, N)                | Topology             | Status
-------------------------------------------------------------------------------------
0    | 15   | [0:1, 0:4, 0:1]                | Top(1, 4, 1)         | 0/64 children
1    | 33   | [1:2, 0:4, 0:1]                | Top(1, 4, 1)         | 0/64 children
2    | 39   | [2:3, 0:4, 0:1]                | Top(1, 4, 1)         | 0/64 children
3    | 57   | [3:4, 0:4, 0:1]                | Top(1, 4, 1)         | 0/64 children
16   | 63   | [0:1, 0:4, 1:2]                | Top(1, 4, 1)         | 0/64 children
17   | 17   | [1:2, 0:4, 1:2]                | Top(1, 4, 1)         | 0/64 children
18   | 23   | [2:3, 0:4, 1:2]                | Top(1, 4, 1)         | 0/64 children
19   | 41   | [3:4, 0:4, 1:2]                | Top(1, 4, 1)         | 0/64 children
32   | 47   | [0:1, 0:4, 2:3]     

In [5]:
S = SliceIR_tree(root)

In [6]:
slice_root = S.root

In [7]:
dot_graph = generate_dot_source(S)

In [8]:
dot_data = generate_dot_source(S)
print(dot_data) 
open("debug_tree.dot", "w").write(dot_data)

digraph SliceIR {
    rankdir=TB;
    splines=polyline;
    node [shape=record, fontname="Courier", style=filled, fillcolor="#f8f9fa"];
    edge [fontname="Courier"];
    node_128111372281776 [label="{ Rank:0 | Depth:0 | child_exec:PARALLEL } | { Shape:(4, 4, 4) | Slice:[0:4, 0:4, 0:4] | Offset:(0, 0, 0) }"];
    node_128111372281776 -> node_128111372137920;
    node_128111372137920 [label="{ Rank:1 | Depth:1 | child_exec:SERIAL } | { Shape:(1, 4, 1) | Slice:[1:2, 0:4, 2:3] | Offset:(1, 0, 2) }"];
    node_128111372281776 -> node_128111372138160;
    node_128111372138160 [label="{ Rank:7 | Depth:1 | child_exec:SERIAL } | { Shape:(1, 4, 1) | Slice:[2:3, 0:4, 2:3] | Offset:(2, 0, 2) }"];
    node_128111372281776 -> node_128111372264912;
    node_128111372264912 [label="{ Rank:9 | Depth:1 | child_exec:SERIAL } | { Shape:(1, 4, 1) | Slice:[3:4, 0:4, 3:4] | Offset:(3, 0, 3) }"];
    node_128111372281776 -> node_128111372736848;
    node_128111372736848 [label="{ Rank:15 | Depth:1 | child_ex

3397